This notebook reuses most of my Topics identification model from Feedback 1. Since the effectiveness of Lead and Position are coupled to the Prompt and the remaining labels are coupled to Position directly or indirectly, using Prompts as an input could provide additional useful context to the models. The model uses all essays from Feedback 1 and uses the 15 topics (I think) provided in it. It can be rebuilt with upgraded versions of the corresponding libraries and this new dataset, if required.


The Topic CSV file is just a map of predicted topics to actual prompt questions that I got from searching some keywords. 

In [ ]:
!pip uninstall -y scikit-learn sklearn
!pip install scikit-learn==0.24.2

In [ ]:
!conda install ../input/offline-downloads-topics-identification/btopic_whls/hdbscan-0.8.28-py37hb1e94ed_1.tar.bz2 --use-index-cache
!pip install ../input/offline-downloads-topics-identification/btopic_whls/sentence-transformers-2.2.0.tar.gz --no-index --find-links=../input/offline-downloads-topics-identification/btopic_whls
!pip install ../input/offline-downloads-topics-identification/btopic_whls/bertopic-0.9.4-py2.py3-none-any.whl --no-index --find-links=../input/offline-downloads-topics-identification/btopic_whls --no-dependencies

In [ ]:
from bertopic import BERTopic
import pandas as pd, numpy as np, glob
from tqdm import tqdm

In [ ]:
topic_model = BERTopic.load("../input/offline-downloads-topics-identification/fb_topic_model")

In [ ]:
fls = glob.glob("../input/feedback-prize-effectiveness/train/*.txt")
df = []
for fl in tqdm(fls):
    eid = fl.split("/")[-1].split(".")[0]
    with open(fl) as f:
        txt = f.read()
    df.append([eid, txt])

df = pd.DataFrame(df, columns=["id", "txt"])

In [ ]:
preds, pred_probs = topic_model.transform(df["txt"].tolist())
df["topic"] = preds
#df["pred"] = preds
#df["prob"] = pred_probs

In [ ]:
topicdf = pd.read_csv("../input/topics-to-long-questions/topic_model_metadata_orig.csv")
topicdf = topicdf[["topic", "desc"]]
df = pd.merge(df, topicdf, how="left", on="topic")

In [ ]:
df.to_csv("fb2_train_topics.csv", index=False)

In [ ]:
!cp ../input/topics-to-long-questions/topic_model_metadata_orig.csv . 